In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy.io
import math
import matplotlib.pyplot as plt
import subprocess
from PIL import Image

In [2]:
def softmax(Z):
    
    Zshift = Z - np.max(Z)
    
    t = np.exp(Zshift)
    a = np.divide(t, (np.sum(t, axis=0, keepdims=True)))
    
    return a, Z

In [19]:
def softmax_backward_extended(dA, cache):
    """Unvectorized computation of the gradient of softmax.
 
    z: (T, 1) column array of input values.
 
    Returns D (T, T) the Jacobian matrix of softmax(z) at the given z. D[i, j]
    is DjSi - the partial derivative of Si w.r.t. input j.
    """
    Z = cache    
    a, _ = softmax(Z)
    
    assert(Z.shape == dA.shape)
    
    noOfClass, m = Z.shape
    dJdZ = np.zeros([noOfClass, m])
    
    #print(dA[:,1])
    
    for k in range(m):
        dAdZ = np.zeros((noOfClass, noOfClass))
        
        for i in range(noOfClass):
            for j in range(noOfClass):
                dAdZ[i, j] = a[i, k] * ((i == j) - a[j, k])

        D = -np.outer(a[:, k], a[:, k]) + np.diag(a[:, k].flatten())
        
        if (np.sum(dAdZ) - np.sum(D)) > 1e-16:
            print("too big")
        assert(D.shape ==  dAdZ.shape)
        
        #print((dA[:,k].reshape(noOfClass,1) * D).shape)
        
        new_vector = np.sum (dA[:,k].reshape(noOfClass,1) * D, axis=1, keepdims=True)
        #print(new_vector.shape)
        #new_vector.shape
        #print(dA[:,k].reshape(noOfClass,1).shape)
        #new1_vector = np.sum( dA[:,k].reshape(noOfClass,1) * dAdZ  , axis = 1, keepdims=True)
        #new_vector = dA[:,k].reshape(noOfClass,1) * np.sum(dAdZ, axis = 1, keepdims=True)
        
        #print(new_vector.shape)
        
        if k == 0:
            dJdZ = new_vector

        else:
            dJdZ = np.concatenate((dJdZ, new_vector), axis=1)
            
            

    dJdZa = a + dA*a

    print(dJdZa.shape)
    
    return dJdZ, dJdZa

In [41]:
Z = np.random.rand(10,4000)

Y = np.zeros([10,4000])


Y[np.argmax(Z, axis=0).reshape(1,4000),:]=1
print(np.sum(Y))


dJdZ, dJdZa = softmax_backward_extended(Z, Z)
#print(dJdZa.shape)

40000.0
(10, 4000)


In [31]:
np.sum(dJdZ) - np.sum(dJdZa)

-6296.2517253829465

In [6]:
#D = -np.outer(Z, Z) + np.diag(Z.flatten())
#print(D)